In [ ]:
rm(list = ls(all=TRUE)) # remove all objects
Sys.setlocale("LC_ALL", "zh_CN.UTF-8") # set locale to Chinese
setwd("./") # change working directory

# Data Reading and Preprocessing

## Primitive Variables

### Stock Return

>  Monthly, between the first and last *trading day* of the month (not necessarily beginning and end of the month)

- Read stock return

In [ ]:
# Read the stock data using read.csv
stock <- read.csv("stock.csv", header = TRUE, stringsAsFactors = FALSE, check.names = FALSE, colClasses = c(stkcd = "character", year = "integer", month = "integer"))

index_cols <- c("year", "month")

# Clean column names by removing the suffix (like ".SZ") from stock IDs
colnames(stock)[-1] <- gsub("\\.[A-Z]+$", "", colnames(stock)[-1])

# Identify stock return columns (all except date, year, and month)
stock_id <- colnames(stock)[!(colnames(stock) %in% index_cols)]

# Convert all stock ID columns to numeric
for (id in stock_id) {
    stock[[id]] <- as.numeric(stock[[id]])
}

# Display the first few rows of the data
head(stock)

- Read listing date

In [ ]:
# Read the list_date.csv file
list_time <- read.csv("list_time.csv", header = TRUE, stringsAsFactors = FALSE, colClasses = c(stkcd = "character", year = "integer", month = "integer"))

# Clean stock column by removing the suffix (like ".SZ") from stock IDs
list_time$stkcd <- gsub("\\.[A-Z]+$", "", list_time$stkcd)

head(list_time)

- For each stock, remove observations within the 1 year period after the listing date

In [ ]:
# Create a copy of the stock dataframe
stock_filtered <- stock

# Track statistics
affected_stocks <- 0
removed_observations <- 0

# For each stock, remove observations where it has been listed for less than 1 year
for (id in stock_id) {
    # Find the stock in list_time
    idx <- which(list_time$stkcd == id)
    
    if (length(idx) > 0) {
        # Get the listing year and month
        listing_year <- list_time$year[idx]
        listing_month <- list_time$month[idx]
        
        # For each row in stock_filtered
        for (row in 1:nrow(stock_filtered)) {
            current_year <- stock_filtered$year[row]
            current_month <- stock_filtered$month[row]
            
            # Check if this observation is within 1 year of listing
            years_difference <- current_year - listing_year
            months_difference <- current_month - listing_month
            total_months_difference <- years_difference * 12 + months_difference
            
            if (total_months_difference < 12) {
                # Count if this cell has data (not NA) before setting it to NA
                if (!is.na(stock_filtered[row, id])) {
                    removed_observations <- removed_observations + 1
                    if (sum(!is.na(stock_filtered[, id])) > 0) {
                        affected_stocks <- affected_stocks + 1
                    }
                }
                
                # Remove this observation
                stock_filtered[row, id] <- NA
            }
        }
    }
}

# Count unique affected stocks (the previous count might double-count)
affected_stocks <- sum(sapply(stock_id, function(id) {
    any(is.na(stock_filtered[, id])) & any(!is.na(stock[, id]))
}))

# Print summary
cat("Removed observations for", affected_stocks, "stocks that had been listed for less than 1 year\n")
cat("Total observations removed:", removed_observations, "\n")

# Update the stock dataframe
stock <- stock_filtered

head(stock)

- Remove stocks with too many missing values under the prioritized criteria:
  1. Remove as fewer stocks as possible
  2. Common time scope of the remaining observations should be closer to latest as possible
  3. Common time scope of the remaining observations should be as long as possible

In [ ]:
# Create a copy of the stock dataframe
stock_filtered <- stock

# Create a matrix indicating non-NA values
has_data <- !is.na(stock_filtered[, stock_id])
colnames(has_data) <- stock_id

# Calculate the percentage of non-NA values for each stock
stock_valid_pct <- colMeans(has_data) * 100
names(stock_valid_pct) <- stock_id

# Create a year-month identifier for easier period tracking
stock_filtered$ym <- stock_filtered$year * 100 + stock_filtered$month

# Function to find the common non-NA period for a set of stocks
find_common_period <- function(stocks_subset) {
    # For each period, check if all stocks have data
    all_valid <- apply(has_data[, stocks_subset], 1, all)
    
    # If no common valid dates found, return NULL
    if (!any(all_valid)) {
        return(NULL)
    }
    
    # Find continuous valid periods
    runs <- rle(all_valid)
    valid_run_indices <- which(runs$values)
    
    if (length(valid_run_indices) == 0) {
        return(NULL)
    }
    
    # Find all valid runs and their properties
    valid_periods <- data.frame(
        run_idx = valid_run_indices,
        length = runs$lengths[valid_run_indices],
        stringsAsFactors = FALSE
    )
    
    # Calculate start and end positions for each run
    valid_periods$start_pos <- sapply(valid_periods$run_idx, function(idx) {
        if(idx == 1) 1 else sum(runs$lengths[1:(idx-1)]) + 1
    })
    valid_periods$end_pos <- valid_periods$start_pos + valid_periods$length - 1
    
    # Add start and end year-month
    valid_periods$start_ym <- stock_filtered$ym[valid_periods$start_pos]
    valid_periods$end_ym <- stock_filtered$ym[valid_periods$end_pos]
    
    # Calculate recency score - higher for more recent periods
    max_ym <- max(stock_filtered$ym)
    valid_periods$recency_score <- valid_periods$end_ym / max_ym
    
    # Normalize length
    max_length <- max(valid_periods$length)
    valid_periods$norm_length <- valid_periods$length / max_length
    
    # Combined score: 60% length, 40% recency (favoring recent periods)
    valid_periods$combined_score <- 0.6 * valid_periods$norm_length + 0.4 * valid_periods$recency_score
    
    # Find the period with the highest combined score
    best_idx <- which.max(valid_periods$combined_score)
    
    return(list(
        start_idx = valid_periods$start_pos[best_idx],
        end_idx = valid_periods$end_pos[best_idx],
        length = valid_periods$length[best_idx],
        start_ym = valid_periods$start_ym[best_idx],
        end_ym = valid_periods$end_ym[best_idx],
        combined_score = valid_periods$combined_score[best_idx]
    ))
}

# Initialize variables to store best solution
best_solution <- NULL
best_score <- -Inf

# Try different thresholds to include more or fewer stocks
thresholds <- seq(95, 50, by = -5)

for (threshold in thresholds) {
    # Select stocks with valid data percentage above threshold
    candidate_stocks <- names(stock_valid_pct[stock_valid_pct >= threshold])
    
    # Skip if too few stocks meet the threshold
    if (length(candidate_stocks) < 2) {
        next
    }
    
    # Find common period for these stocks
    period <- find_common_period(candidate_stocks)
    
    # Skip if no common period found
    if (is.null(period)) {
        next
    }
    
    # Total score combines period quality and number of stocks
    # Normalize stock count to give more weight to solutions with more stocks
    stock_ratio <- length(candidate_stocks) / length(stock_id)
    total_score <- period$combined_score + 0.2 * stock_ratio
    
    # Update best solution if better score found
    if (total_score > best_score) {
        best_solution <- list(
            stocks = candidate_stocks,
            start_idx = period$start_idx,
            end_idx = period$end_idx,
            length = period$length,
            start_ym = period$start_ym,
            end_ym = period$end_ym
        )
        best_score <- total_score
    }
}

if (!is.null(best_solution)) {
    # Identify stocks to remove
    stocks_to_remove <- setdiff(stock_id, best_solution$stocks)
    
    # Format year-month for display (YYYY-MM)
    format_ym <- function(ym) {
        year <- floor(ym/100)
        month <- ym %% 100
        return(paste0(year, "-", sprintf("%02d", month)))
    }
    
    # Print summary
    cat("Removing", length(stocks_to_remove), "stocks with excessive NA values:", 
            paste(stocks_to_remove, collapse=", "), "\n")
    cat("Retained", length(best_solution$stocks), "stocks\n")
    cat("Common non-NA period:", format_ym(best_solution$start_ym), "to", 
            format_ym(best_solution$end_ym), "(", best_solution$length, "periods )\n")
    
    # Update stock_id to the selected stocks
    stock_id <- best_solution$stocks
    
    # Filter the dataframe to keep only the selected stocks and date range
    stock_filtered <- stock_filtered[best_solution$start_idx:best_solution$end_idx, c("year", "month", best_solution$stocks)]
    
    # Verify no NA values in the filtered dataset
    cat("NA values in filtered dataset:", sum(is.na(stock_filtered)), "\n")
} else {
    cat("Could not find a common non-NA period for any subset of stocks\n")
}

# Update the stock dataframe
stock <- stock_filtered

head(stock)

- Trim extreme values and replace missing values

In [ ]:
# Create a copy of the stock dataframe
stock_filtered <- stock

# Make sure data is ordered by year and month
stock_filtered <- stock_filtered[order(stock_filtered$year, stock_filtered$month),]

# Initialize counters for reporting
total_outliers <- 0
total_na_replacements <- 0
outliers_by_stock <- numeric(length(stock_id))
names(outliers_by_stock) <- stock_id

# Function to handle outliers with various methods
replace_outliers <- function(x, method = "interp") {
    # Create a copy of the vector with identified outlier positions as NA
    x_replaced <- x
    
    # Skip if no NA values (no outliers to replace)
    if (!any(is.na(x_replaced))) return(x_replaced)
    
    # Apply chosen replacement method
    if (method == "interp") {
        # Linear interpolation
        na_idx <- which(is.na(x_replaced))
        for (idx in na_idx) {
            # Find nearest non-NA values before and after
            before_idx <- idx - 1
            while(before_idx > 0 && is.na(x_replaced[before_idx])) before_idx <- before_idx - 1
            
            after_idx <- idx + 1
            while(after_idx <= length(x_replaced) && is.na(x_replaced[after_idx])) after_idx <- after_idx + 1
            
            # Interpolate if both bounds exist
            if (before_idx > 0 && after_idx <= length(x_replaced)) {
                x_replaced[idx] <- x_replaced[before_idx] + 
                    (x_replaced[after_idx] - x_replaced[before_idx]) * 
                    (idx - before_idx) / (after_idx - before_idx)
            } else if (before_idx > 0) {
                # If only before exists, use that value
                x_replaced[idx] <- x_replaced[before_idx]
            } else if (after_idx <= length(x_replaced)) {
                # If only after exists, use that value
                x_replaced[idx] <- x_replaced[after_idx]
            } else {
                # Fallback to zero if no reference points
                x_replaced[idx] <- 0
            }
        }
    } else if (method == "median") {
        # Replace with rolling median (window of 5 months)
        window_size <- 5
        na_idx <- which(is.na(x_replaced))
        for (idx in na_idx) {
            # Define window bounds
            start <- max(1, idx - floor(window_size/2))
            end <- min(length(x), idx + floor(window_size/2))
            
            # Get values in window excluding the current NA
            window_values <- x[start:end]
            window_values <- window_values[!is.na(window_values)]
            
            if (length(window_values) > 0) {
                x_replaced[idx] <- median(window_values)
            } else {
                x_replaced[idx] <- 0 # Fallback if all window values are NA
            }
        }
    } else if (method == "zero") {
        # Simple zero replacement
        x_replaced[is.na(x_replaced)] <- 0
    }
    
    return(x_replaced)
}

# Clean each return series
for (id in stock_id) {
    # Use rolling window for adaptive outlier detection
    window_size <- 12 # 12-month rolling window (adjusted from 20 days)
    n <- length(stock_filtered[,id])
    
    for (i in window_size:n) {
        # Define the rolling window
        window_start <- i - window_size + 1
        window_data <- stock_filtered[window_start:i, id]
        current_value <- window_data[window_size]
        
        # Skip if the current value is NA
        if (is.na(current_value)) next
        
        # Calculate robust statistics from the window (excluding current point)
        window_prev <- window_data[-window_size]
        window_median <- median(window_prev, na.rm = TRUE)
        window_mad <- mad(window_prev, na.rm = TRUE)
        
        # Skip if median or MAD is NA or MAD is 0 (no variability)
        if (is.na(window_median) || is.na(window_mad) || window_mad == 0) next
        
        # Use median absolute deviation (MAD) for robust outlier detection
        mad_threshold <- 5 # 5 MADs is a common threshold for extreme outliers
        
        # Check if the current value is an outlier
        if (abs(current_value - window_median) > mad_threshold * window_mad) {
            # Mark as outlier
            stock_filtered[i, id] <- NA
            outliers_by_stock[id] <- outliers_by_stock[id] + 1
            total_outliers <- total_outliers + 1
        }
    }
    
    # Replace outliers and NAs with chosen method
    if (any(is.na(stock_filtered[,id]))) {
        na_count_before <- sum(is.na(stock_filtered[,id]))
        stock_filtered[,id] <- replace_outliers(stock_filtered[,id], method = "interp")
        na_count_after <- sum(is.na(stock_filtered[,id]))
        total_na_replacements <- total_na_replacements + (na_count_before - na_count_after)
    }
}

# Print report
cat("Extreme Value Removal Report:\n")
cat("----------------------------\n")
cat("Total outliers identified and removed:", total_outliers, "\n")
cat("Total missing values replaced:", total_na_replacements, "\n\n")

# Print stocks with the most outliers
if (total_outliers > 0) {
    cat("Stocks with outliers:\n")
    for (id in stock_id) {
        if (outliers_by_stock[id] > 0) {
            cat("  -", id, ":", outliers_by_stock[id], "outliers\n")
        }
    }
}

# Use the cleaned data for further analysis
stock <- stock_filtered

head(stock)

- Detrend the return series using HP filter

In [ ]:
library(mFilter)

# Create a new dataframe to store the cyclical components with year and month
cycle <- data.frame(year = stock$year, month = stock$month)

# Ensure the data is sorted chronologically
cycle <- cycle[order(cycle$year, cycle$month), ]

# Apply HP filter to each stock return series and extract the cyclical component
for (id in stock_id) {
    # Apply HP filter with lambda=14400 (commonly used for monthly financial data)
    hp_filter <- hpfilter(stock[,id], freq=14400)
    
    # Extract the cyclical component and add to cycle dataframe
    cycle[,id] <- hp_filter$cycle
}

# Display the first few rows of the cycle dataframe
head(cycle)

### VIX

> Daily, close value of the day

In [ ]:
# Read the VIX data using read.csv
vix <- read.csv("vix.csv", header = TRUE, stringsAsFactors = FALSE, check.names = FALSE)

# Convert date column to Date format using base R syntax
vix$date <- as.Date(vix$date, format = "%m/%d/%Y")

# Filter to keep only date and close columns
vix <- vix[, c("date", "close")]

# Sort data by date to ensure chronological order
vix <- vix[order(vix$date), ]

# Identify gaps in the time series
date_range <- range(vix$date)
complete_dates <- seq(date_range[1], date_range[2], by = "day")
missing_dates <- complete_dates[!complete_dates %in% vix$date]

# Report missing dates
cat("Found", length(missing_dates), "missing dates in VIX time series\n")

# Create a complete dataframe with all dates
complete_vix <- data.frame(date = complete_dates)

# Merge with actual data, which will result in NA for missing dates
vix_filled <- merge(complete_vix, vix, by = "date", all.x = TRUE)

# Fill missing values using linear interpolation
library(zoo)
vix_filled$close <- na.approx(vix_filled$close, na.rm = FALSE)

# Handle any remaining NAs at the beginning or end 
# that couldn't be interpolated
if (any(is.na(vix_filled$close))) {
    # Fill NAs at beginning with next observation
    vix_filled$close <- na.locf(vix_filled$close, fromLast = TRUE, na.rm = FALSE)
    
    # Fill NAs at end with last observation
    vix_filled$close <- na.locf(vix_filled$close, na.rm = FALSE)
    
    cat("Filled remaining edge values using last/next observation carrying\n")
}

# Update the vix dataframe
vix <- vix_filled

# Quick summary
cat("VIX data now has", nrow(vix), "daily observations with no gaps\n")

# Display the first few rows of the filtered VIX data
head(vix)

### GFCF

> Monthly, Global Financial Cycle Factor, Common factor across world risky asset prices as in Miranda-Agrippino and Rey (2020), *"US Monetary Policy and the Global Financial Cycle"*

In [ ]:
# Read the GFCF data
gfcf <- read.csv("gfcf.csv", header = TRUE, stringsAsFactors = FALSE, colClasses = c(year = "integer", month = "integer"))

# Sort by date to ensure chronological order
gfcf <- gfcf[order(gfcf$year, gfcf$month), ]

# Check for gaps in year-month sequence
min_year <- min(gfcf$year)
max_year <- max(gfcf$year)
min_month <- min(gfcf$month[gfcf$year == min_year])
max_month <- max(gfcf$month[gfcf$year == max_year])

# Create complete sequence of year-month combinations
all_dates <- expand.grid(
    year = min_year:max_year,
    month = 1:12
)
all_dates <- all_dates[
    (all_dates$year > min_year | all_dates$month >= min_month) &
    (all_dates$year < max_year | all_dates$month <= max_month),
]
all_dates <- all_dates[order(all_dates$year, all_dates$month), ]

# Merge with actual data to identify gaps
complete_gfcf <- merge(all_dates, gfcf, by = c("year", "month"), all.x = TRUE)

# Check for missing values after merge
missing_count <- sum(is.na(complete_gfcf$gfcf))

if (missing_count > 0) {
    cat("Found", missing_count, "missing year-month combinations in the GFCF time series\n")
    
    # Fill missing values using interpolation
    library(zoo)
    
    # Create a time index for interpolation (decimal year)
    complete_gfcf$time_idx <- complete_gfcf$year + (complete_gfcf$month - 1) / 12
    
    # Interpolate missing values
    complete_gfcf$gfcf <- na.approx(complete_gfcf$gfcf, x = complete_gfcf$time_idx, na.rm = FALSE)
    
    # Fill any remaining NAs at the beginning or end
    complete_gfcf$gfcf <- na.locf(complete_gfcf$gfcf, fromLast = TRUE, na.rm = FALSE)
    complete_gfcf$gfcf <- na.locf(complete_gfcf$gfcf, na.rm = FALSE)
    
    # Remove the time index column (was just for interpolation)
    complete_gfcf$time_idx <- NULL
} else {
    cat("No gaps detected in the GFCF year-month sequence\n")
}

# Update the gfcf dataframe
gfcf <- complete_gfcf[order(complete_gfcf$year, complete_gfcf$month), ]

# Verify no more missing values
final_missing <- sum(is.na(gfcf$gfcf))
cat("Missing values after filling:", final_missing, "\n")

# Display summary statistics
cat("\nGFCF data summary:\n")
summary(gfcf$gfcf)

# Display the first few rows
head(gfcf)

### Capital Inflow

> Quarterly, normalized by quarter GDP value in USD

In [ ]:
# Read the capital inflow data
inflow <- read.csv("channel_inflow.csv", header = TRUE, stringsAsFactors = FALSE, 
                   colClasses = c(year = "integer", quarter = "integer"))

# Sort by date to ensure chronological order
inflow <- inflow[order(inflow$year, inflow$quarter), ]

# Check for gaps in year-quarter sequence
min_year <- min(inflow$year)
max_year <- max(inflow$year)
min_quarter <- min(inflow$quarter[inflow$year == min_year])
max_quarter <- max(inflow$quarter[inflow$year == max_year])

# Create complete sequence of year-quarter combinations
all_dates <- expand.grid(
    year = min_year:max_year,
    quarter = 1:4
)
all_dates <- all_dates[
    (all_dates$year > min_year | all_dates$quarter >= min_quarter) &
    (all_dates$year < max_year | all_dates$quarter <= max_quarter),
]
all_dates <- all_dates[order(all_dates$year, all_dates$quarter), ]

# Add month column for each quarter's end (Q1=3, Q2=6, Q3=9, Q4=12)
all_dates$month <- all_dates$quarter * 3

# Merge with actual data to identify gaps
complete_inflow <- merge(all_dates, inflow, by = c("year", "quarter"), all.x = TRUE)

# Check for missing values after merge
missing_count <- sum(is.na(complete_inflow$inflow))

if (missing_count > 0) {
    cat("Found", missing_count, "missing year-quarter combinations in the inflow time series\n")
    
    # Fill missing values using interpolation
    library(zoo)
    
    # Create a time index for interpolation (decimal year)
    complete_inflow$time_idx <- complete_inflow$year + (complete_inflow$quarter - 1) / 4
    
    # Interpolate missing values
    complete_inflow$inflow <- na.approx(complete_inflow$inflow, x = complete_inflow$time_idx, na.rm = FALSE)
    
    # Fill any remaining NAs at the beginning or end
    complete_inflow$inflow <- na.locf(complete_inflow$inflow, fromLast = TRUE, na.rm = FALSE)
    complete_inflow$inflow <- na.locf(complete_inflow$inflow, na.rm = FALSE)
    
    # Remove the time index column (was just for interpolation)
    complete_inflow$time_idx <- NULL
} else {
    cat("No gaps detected in the inflow year-quarter sequence\n")
}

# Update the inflow dataframe
inflow <- complete_inflow[order(complete_inflow$year, complete_inflow$quarter), ]

# Verify no more missing values
final_missing <- sum(is.na(inflow$inflow))
cat("Missing values after filling:", final_missing, "\n")

# Display summary statistics
cat("\nCapital inflow data summary:\n")
summary(inflow$inflow)

# Display the first few rows
head(inflow)

### Economic Policy Uncertainty (EPU)

> Monthly

In [ ]:
# Read the EPU data
epu <- read.csv("channel_epu.csv", header = TRUE, stringsAsFactors = FALSE, 
               colClasses = c(year = "integer", month = "integer"))

# Add quarter column
epu$quarter <- as.integer(ceiling(epu$month / 3))

# Sort by date to ensure chronological order
epu <- epu[order(epu$year, epu$quarter, epu$month), ]

# Check for missing values in epu
cat("Missing values in EPU:", sum(is.na(epu$epu)), "\n")

# Handle any missing values if necessary
if(sum(is.na(epu$epu)) > 0) {
  # Use linear interpolation for missing values
  library(zoo)
  epu$epu <- na.approx(epu$epu, na.rm = FALSE)
  # Handle any remaining edge cases
  epu$epu <- na.locf(epu$epu, fromLast = TRUE, na.rm = FALSE)
  epu$epu <- na.locf(epu$epu, na.rm = FALSE)
}

# Calculate quarterly average EPU values
quarterly_epu <- aggregate(epu ~ year + quarter, data = epu, FUN = mean)

# Add month column (last month of each quarter)
quarterly_epu$month <- quarterly_epu$quarter * 3

# Calculate log of quarterly average EPU
quarterly_epu$log_epu <- log(quarterly_epu$epu)

# Select only the needed columns
epu <- quarterly_epu[, c("year", "quarter", "month", "log_epu")]

# Sort by year, quarter
epu <- epu[order(epu$year, epu$quarter), ]

# Display summary
cat("EPU data processed with", nrow(epu), "quarterly observations\n")
summary(epu$log_epu)

# Display the first few rows
head(epu)

### Control

> Quarterly, quarter-end month observation

#### Micro

- Read the micro part of control variable and balance the panel

- Dataset has already received the following cleansing before input:
  1. Removed:
     - stocks from the financial sector
     - stocks incurred ST/ST*/PT during observation period
  2. Winsorized: 5% on both ends

In [ ]:
# Read the control_micro data
control_micro <- read.csv("control_micro.csv", header = TRUE, stringsAsFactors = FALSE, 
        colClasses = c(stkcd = "character", year = "integer", quarter = "integer", month = "integer"))

index_cols <- c("stkcd", "year", "quarter", "month")
var_cols <- c("Size", "Lev", "ROA", "BM", "Board", "Top1", "SOE")

control_micro <- control_micro[, c(index_cols, var_cols)]

# Find the earliest and latest year-quarter combinations
min_year <- min(control_micro$year)
min_quarter <- min(control_micro$quarter[control_micro$year == min_year])
max_year <- max(control_micro$year)
max_quarter <- max(control_micro$quarter[control_micro$year == max_year])

cat("Date range for all stocks:", min_year, "Q", min_quarter, "to", max_year, "Q", max_quarter, "\n")

# Create a complete sequence of year-quarter combinations
all_dates <- expand.grid(
  year = min_year:max_year,
  quarter = 1:4
)

# Filter to keep only dates within the identified range
all_dates <- all_dates[
  (all_dates$year > min_year | all_dates$quarter >= min_quarter) &
  (all_dates$year < max_year | all_dates$quarter <= max_quarter),
]
all_dates <- all_dates[order(all_dates$year, all_dates$quarter), ]

# Add month column based on quarter (Q1=3, Q2=6, Q3=9, Q4=12)
all_dates$month <- all_dates$quarter * 3

# Get unique stock codes
unique_stocks <- unique(control_micro$stkcd)

# Create an empty dataframe to hold the balanced panel
balanced_control_micro <- data.frame()

# For each stock, add missing dates
for (stock in unique_stocks) {
  # Get data for this stock
  stock_data <- control_micro[control_micro$stkcd == stock, ]
  
  # Create stock-specific complete date sequence
  stock_dates <- data.frame(
  stkcd = stock,
  year = all_dates$year,
  quarter = all_dates$quarter,
  month = all_dates$month
  )
  
  # Merge with actual data (will add NA for missing dates)
  complete_stock_data <- merge(
  stock_dates,
  stock_data,
  by = index_cols,
  all.x = TRUE
  )
  
  # Add to the balanced panel
  balanced_control_micro <- rbind(balanced_control_micro, complete_stock_data)
}

# Sort the balanced panel
balanced_control_micro <- balanced_control_micro[order(balanced_control_micro$stkcd, balanced_control_micro$year, balanced_control_micro$quarter), c(index_cols, var_cols)]

# Display summary statistics
cat("Original data had", nrow(control_micro), "observations\n")
cat("Balanced panel has", nrow(balanced_control_micro), "observations\n")
cat("Added", nrow(balanced_control_micro) - nrow(control_micro), "missing observations\n")

# Calculate how many stocks have missing observations
stocks_with_gaps <- sapply(unique_stocks, function(stock) {
  stock_data <- control_micro[control_micro$stkcd == stock, ]
  expected_obs <- nrow(all_dates)
  actual_obs <- nrow(stock_data)
  return(expected_obs != actual_obs)
})

cat("Number of stocks with time gaps:", sum(stocks_with_gaps), "out of", length(unique_stocks), "\n")

# Update the control_micro dataframe
control_micro <- balanced_control_micro

# Display the first few rows
head(control_micro)

In [ ]:
# Read industry classification data
ind_type <- read.csv("ind_type.csv", header = TRUE, stringsAsFactors = FALSE, 
                    colClasses = c(stkcd = "character", year = "integer"))

# Ensure stock code format consistency for merging
if(any(grepl("\\.", ind_type$stkcd))) {
  ind_type$stkcd <- gsub("\\.[A-Z]+$", "", ind_type$stkcd)
}

# Check for duplicate entries
duplicates <- ind_type[duplicated(ind_type[, c("year", "stkcd")]) | 
                      duplicated(ind_type[, c("year", "stkcd")], fromLast = TRUE), ]
if(nrow(duplicates) > 0) {
  cat("Warning: Found", nrow(duplicates)/2, "duplicate year-stock combinations in industry data\n")
}

# Merge with control_micro using inner join
control_micro_with_ind <- merge(
  control_micro,
  ind_type,
  by = c("year", "stkcd"),
  all.x = FALSE
)

# Check dimensions before and after merge
cat("Rows in control_micro before merge:", nrow(control_micro), "\n")
cat("Rows after merge with industry data:", nrow(control_micro_with_ind), "\n")
cat("Stocks with missing industry information:", 
    length(setdiff(unique(control_micro$stkcd), unique(control_micro_with_ind$stkcd))), "\n")

# Convert industry codes to factors for regression analysis
control_micro_with_ind$ind <- factor(control_micro_with_ind$ind)

# Update the control_micro dataframe
control_micro <- control_micro_with_ind

# Display the first few rows of the merged data
head(control_micro)

- Treat the missing values

In [ ]:
library(zoo)

# Check how many missing values we have in each column
missing_values <- colSums(is.na(control_micro))
cat("Missing values per column:\n")
print(missing_values)

# Calculate percentage of missing values by column
pct_missing <- round(missing_values / nrow(control_micro) * 100, 2)
cat("\nPercentage of missing values:\n")
print(pct_missing)

# For visualization, get missing pattern by stock and time
missing_by_stock <- tapply(rowSums(is.na(control_micro[, var_cols])), 
                          control_micro$stkcd, mean)
missing_by_time <- aggregate(rowSums(is.na(control_micro[, var_cols])), 
                            by = list(year = control_micro$year, 
                                     quarter = control_micro$quarter), 
                            mean)

# Impute missing values based on variable characteristics
control_micro_imputed <- control_micro

# Approach for each variable:
for (id in unique(control_micro$stkcd)) {
  # Get data for this stock only
  stock_data <- control_micro[control_micro$stkcd == id, ]
  stock_data <- stock_data[order(stock_data$year, stock_data$quarter), ]
  
  for (var in var_cols) {
    # Skip if no missing values for this variable and stock
    if (!any(is.na(stock_data[, var]))) next
    
    # Choose imputation method based on variable type
    if (var %in% c("Size", "Lev", "ROA", "BM")) {
      # For financial metrics that change gradually, use interpolation
      stock_data[, var] <- na.approx(stock_data[, var], na.rm = FALSE)
      
      # Handle any remaining NAs at the beginning with next observation
      if (any(is.na(stock_data[, var]))) {
        stock_data[, var] <- na.locf(stock_data[, var], fromLast = TRUE, na.rm = FALSE)
        
        # Handle any remaining NAs at the end with last observation
        stock_data[, var] <- na.locf(stock_data[, var], na.rm = FALSE)
      }
    } else if (var %in% c("Board", "Top1", "SOE", "ind1", "ind2")) {
      # For governance metrics that change infrequently, use LOCF
      stock_data[, var] <- na.locf(stock_data[, var], na.rm = FALSE)
      
      # If there are still NAs at the beginning, use NOCB
      if (any(is.na(stock_data[, var]))) {
        stock_data[, var] <- na.locf(stock_data[, var], fromLast = TRUE, na.rm = FALSE)
      }
      
      # For any remaining NAs (if entire series is NA), use cross-sectional median
      if (any(is.na(stock_data[, var]))) {
        # Get the median value from the whole dataset for this variable
        var_median <- median(control_micro[, var], na.rm = TRUE)
        stock_data[is.na(stock_data[, var]), var] <- var_median
      }
    }
  }
  
  # Update the imputed dataset with this stock's data
  control_micro_imputed[control_micro_imputed$stkcd == id, ] <- stock_data
}

# Check if any missing values remain
missing_after <- colSums(is.na(control_micro_imputed))
cat("\nMissing values after imputation:\n")
print(missing_after)

# Display summary statistics before and after imputation
cat("\nSummary before imputation:\n")
print(summary(control_micro[, var_cols]))

cat("\nSummary after imputation:\n")
print(summary(control_micro_imputed[, var_cols]))

# Update the control_micro dataframe
control_micro <- control_micro_imputed

# Display the first few rows of the imputed data
head(control_micro)

#### Macro

- Read the macro part of control variable

In [ ]:
# Read the control_macro data
control_macro <- read.csv("control_macro.csv", header = TRUE, stringsAsFactors = FALSE, 
              colClasses = c(year = "integer", quarter = "integer", month = "integer"))

index_cols <- c("year", "quarter", "month")
var_cols <- c("gdp_growth_rate", "m2_growth_rate")

# Sort by date to ensure chronological order
control_macro <- control_macro[order(control_macro$year, control_macro$quarter), ][, c(index_cols, var_cols)]

# Find the earliest and latest year-quarter combinations
min_year <- min(control_macro$year)
min_quarter <- min(control_macro$quarter[control_macro$year == min_year])
max_year <- max(control_macro$year)
max_quarter <- max(control_macro$quarter[control_macro$year == max_year])

cat("Date range:", min_year, "Q", min_quarter, "to", max_year, "Q", max_quarter, "\n")

# Create a complete sequence of year-quarter combinations
all_dates <- expand.grid(
  year = min_year:max_year,
  quarter = 1:4
)

# Filter to keep only dates within the identified range
all_dates <- all_dates[
  (all_dates$year > min_year | all_dates$quarter >= min_quarter) &
  (all_dates$year < max_year | all_dates$quarter <= max_quarter),
]
all_dates <- all_dates[order(all_dates$year, all_dates$quarter), ]

# Add month column (last month of each quarter)
all_dates$month <- all_dates$quarter * 3

# Merge with actual data (will add NA for missing dates)
complete_control_macro <- merge(
  all_dates,
  control_macro,
  by = index_cols,
  all.x = TRUE
)

# Handle merged month columns if they exist
if ("month.y" %in% colnames(complete_control_macro)) {
  # Use month.y values where available, otherwise use month.x
  complete_control_macro$month <- ifelse(
    is.na(complete_control_macro$month.y),
    complete_control_macro$month.x,
    complete_control_macro$month.y
  )
  
  # Remove the extra month columns
  complete_control_macro$month.x <- NULL
  complete_control_macro$month.y <- NULL
}

# Count missing periods
non_date_cols <- setdiff(names(complete_control_macro), index_cols)
missing_periods <- sum(apply(complete_control_macro[, non_date_cols, drop=FALSE], 1, function(x) all(is.na(x))))

# Report results
cat("Found", missing_periods, "missing year-quarter combinations\n")
if (missing_periods > 0) {
  cat("Added missing periods with NA values for observations\n")
}

# Update the control_macro dataframe
control_macro <- complete_control_macro[order(complete_control_macro$year, complete_control_macro$quarter), ]

# Display the first few rows
head(control_macro)

- Treat the missing values

In [ ]:
# Check current missing values in control_macro data
missing_values <- colSums(is.na(control_macro))
cat("Missing values in control_macro:\n")
print(missing_values)

# Impute missing values for each economic variable
library(zoo)
library(forecast)

# Create a time index for better time series handling
control_macro$time_idx <- control_macro$year + (control_macro$quarter - 1) / 4

# Sort data by time
control_macro <- control_macro[order(control_macro$time_idx), ]

# Function to impute missing economic time series data
impute_economic_ts <- function(x, time_idx) {
    # If no missing values, return as is
    if(!any(is.na(x))) return(x)
    
    # For short gaps (1-2 periods), use spline interpolation
    x_spline <- na.spline(x, na.rm = FALSE)
    
    # For remaining gaps, try time series methods
    if(any(is.na(x_spline))) {
        # Create a time series object
        ts_data <- ts(x, frequency = 4)  # Quarterly data has frequency 4
        
        # Simple imputation with seasonal components
        x_seadec <- na.seadec(ts_data, algorithm = "interpolation")
        
        # Replace values still missing after spline with these estimates
        na_idx <- which(is.na(x_spline))
        x_spline[na_idx] <- x_seadec[na_idx]
        
        # If still have missing values (e.g., at edges), use ARIMA forecasting
        if(any(is.na(x_spline))) {
            # First fill leading NAs with the first non-NA value
            if(is.na(x_spline[1])) {
                first_valid <- min(which(!is.na(x_spline)))
                x_spline[1:first_valid-1] <- x_spline[first_valid]
            }
            
            # Then fill trailing NAs with the last non-NA value
            if(is.na(x_spline[length(x_spline)])) {
                last_valid <- max(which(!is.na(x_spline)))
                x_spline[last_valid+1:length(x_spline)] <- x_spline[last_valid]
            }
        }
    }
    
    return(x_spline)
}

# Apply imputation to each economic variable
control_macro$gdp_growth_rate <- impute_economic_ts(control_macro$gdp_growth_rate, control_macro$time_idx)
control_macro$m2_growth_rate <- impute_economic_ts(control_macro$m2_growth_rate, control_macro$time_idx)

# Remove the time index after imputation
control_macro$time_idx <- NULL

# Check if imputation was successful
missing_after <- colSums(is.na(control_macro))
cat("\nMissing values after imputation:\n")
print(missing_after)

# Verify the imputed data makes economic sense
cat("\nSummary of imputed economic variables:\n")
cat("GDP growth rate summary:\n")
print(summary(control_macro$gdp_growth_rate))
cat("\nM2 growth rate summary:\n")
print(summary(control_macro$m2_growth_rate))

# Display the first few rows to check for patterns
head(control_macro)

#### Merge Micro & Macro

In [ ]:
# Merge the control_micro and control_macro datasets
# This will add macro variables to each stock in each time period

# First, check dimensions of input datasets
cat("Control_micro dimensions:", dim(control_micro)[1], "observations,", dim(control_micro)[2], "variables\n")
cat("Control_macro dimensions:", dim(control_macro)[1], "observations,", dim(control_macro)[2], "variables\n")

# Perform the merge using merge() function with an inner join approach
control <- merge(
    control_micro,  # Stock-level micro controls
    control_macro,  # Time-level macro controls
    by = c("year", "quarter", "month"),  # Join keys
    all.x = FALSE,  # Inner join to keep only matched records
    all.y = FALSE
)

# Verify the structure of the merged dataset
cat("\nMerged control dataset dimensions:", dim(control)[1], "observations,", dim(control)[2], "variables\n")
cat("Number of stocks in merged dataset:", length(unique(control$stkcd)), "\n")
cat("Time period covered:", min(paste(control$year, "Q", control$quarter)), "to", 
        max(paste(control$year, "Q", control$quarter)), "\n")

# Check for missing values in key columns
missing_counts <- colSums(is.na(control))
if (any(missing_counts > 0)) {
    cat("\nMissing values found in the following columns:\n")
    print(missing_counts[missing_counts > 0])
} else {
    cat("\nNo missing values in the merged dataset\n")
}

# Display the first few rows of the merged dataset
head(control)

## Panel preparation

### Variable Calculation

#### GFC

In [ ]:
# 1. Process VIX for quarter-end values
vix$year <- as.integer(format(vix$date, "%Y"))
vix$month <- as.integer(format(vix$date, "%m"))
vix$quarter <- ceiling(vix$month / 3)

# Find last trading day of each quarter
vix_quarter_end <- aggregate(
    cbind(date, close) ~ year + quarter, 
    data = vix, 
    FUN = function(x) x[which.max(as.Date(x))]
)
vix_quarter_end$month <- vix_quarter_end$quarter * 3  # Last month of quarter

# 2. Process VIX for quarterly averages
vix_quarterly_avg <- aggregate(close ~ year + quarter, data = vix, FUN = mean)
vix_quarterly_avg$month <- vix_quarterly_avg$quarter * 3

# 3. Process GFCF for quarter-end values
gfcf_quarter_end <- gfcf[gfcf$month %in% c(3, 6, 9, 12), ]
gfcf_quarter_end$quarter <- ceiling(gfcf_quarter_end$month / 3)

# Calculate logs and first differences
# 1. VIX quarter-end
vix_quarter_end$log_vix <- log(vix_quarter_end$close)
vix_quarter_end <- vix_quarter_end[order(vix_quarter_end$year, vix_quarter_end$quarter), ]
vix_quarter_end$log_diff <- c(NA, diff(vix_quarter_end$log_vix))

# 2. VIX quarterly average
vix_quarterly_avg$log_vix_avg <- log(vix_quarterly_avg$close)
vix_quarterly_avg <- vix_quarterly_avg[order(vix_quarterly_avg$year, vix_quarterly_avg$quarter), ]
vix_quarterly_avg$log_diff_avg <- c(NA, diff(vix_quarterly_avg$log_vix_avg))

# 3. GFCF quarter-end - CHANGED: using direct difference instead of log difference
gfcf_quarter_end <- gfcf_quarter_end[order(gfcf_quarter_end$year, gfcf_quarter_end$quarter), ]
gfcf_quarter_end$diff_gfcf <- c(NA, diff(gfcf_quarter_end$gfcf))

# Merge indicators using inner joins to keep only periods with all data available
# First merge VIX quarter-end with VIX quarterly average
gfc_indicators <- merge(
    vix_quarter_end[, c("year", "quarter", "month", "log_diff")],
    vix_quarterly_avg[, c("year", "quarter", "log_diff_avg")],
    by = c("year", "quarter"),
    all = FALSE  # Inner join
)

# Then merge with GFCF quarter-end - CHANGED: using diff_gfcf instead of log_diff_gfcf
gfc_indicators <- merge(
    gfc_indicators,
    gfcf_quarter_end[, c("year", "quarter", "diff_gfcf")],
    by = c("year", "quarter"),
    all = FALSE  # Inner join
)

# Sort chronologically
gfc_indicators <- gfc_indicators[order(gfc_indicators$year, gfc_indicators$quarter), ]

# Add lagged values (for t-1 period)
gfc_indicators$log_diff_lag1 <- c(NA, gfc_indicators$log_diff[-nrow(gfc_indicators)])
gfc_indicators$log_diff_avg_lag1 <- c(NA, gfc_indicators$log_diff_avg[-nrow(gfc_indicators)])
gfc_indicators$diff_gfcf_lag1 <- c(NA, gfc_indicators$diff_gfcf[-nrow(gfc_indicators)])

# Add date column for easier merging
gfc_indicators$date <- as.Date(paste(gfc_indicators$year, gfc_indicators$month, "01", sep = "-"))

gfc_indicators <- gfc_indicators[, c("year", "month", "quarter", "log_diff_lag1", "log_diff_avg_lag1", "diff_gfcf_lag1")]

# Remove rows with NA for lagged values (first period)
gfc_indicators <- gfc_indicators[!is.na(gfc_indicators$log_diff_lag1), ]

# Display summary
cat("GFC indicators calculated for", nrow(gfc_indicators), "quarters\n")
head(gfc_indicators)

#### Volatility

- Calculate the volatility as the standard deviation of return rate cycle component within a 48-month (4-year) rolling window

In [ ]:
library(zoo)

# Create a new dataframe to store the volatilities with year and month from cycle
volatility <- data.frame(year = cycle$year, month = cycle$month)

# Set the window size for rolling volatility calculation
window_size <- 48 # 48 months

# Ensure data is sorted chronologically
cycle <- cycle[order(cycle$year, cycle$month), ]
volatility <- volatility[order(volatility$year, volatility$month), ]

# Calculate simple rolling standard deviation for each stock
for (id in stock_id) {
    # Use rollapply with standard deviation function
    vol <- rollapply(cycle[,id], width=window_size, 
                    FUN=function(x) sd(x, na.rm=TRUE), 
                    fill=NA, align="right")
    volatility[,id] <- vol
}

# Remove rows with NA values (first 47 observations due to the rolling window)
volatility <- na.omit(volatility)

# Add quarter column
volatility$quarter <- ceiling(volatility$month / 3)  # Calculate quarter from month

# Reorder columns to put year, quarter, month first, followed by stock columns
volatility <- volatility[, c("year", "quarter", "month", stock_id)]

# Display the first few rows of the volatility dataframe
head(volatility)


- Convert the original volatility table from wide to long form

In [ ]:
library(reshape2)

# Filter the volatility data to keep only quarterly data (March, June, September, December)
volatility_long <- volatility

# Reshape from wide to long format
volatility_molten <- melt(
    volatility_long,
    id.vars = c("year", "month", "quarter"),
    measure.vars = stock_id,
    variable.name = "stkcd",
    value.name = "volatility"
)

# Ensure stkcd is character type for consistent merging
volatility_molten$stkcd <- as.character(volatility_molten$stkcd)

# Create a new dataframe with selected columns
volatility <- volatility_molten[, c("stkcd", "year", "quarter", "month", "volatility")]

# Check for time series continuity
# Count observations per stock
obs_per_stock <- table(volatility$stkcd)
cat("Observations per stock - summary:\n")
print(summary(as.numeric(obs_per_stock)))

# Check if all stocks have the same number of time periods
expected_periods <- length(unique(paste(volatility$year, volatility$month, sep="-")))
cat("\nExpected number of periods per stock:", expected_periods, "\n")
cat("Number of stocks with complete data:", sum(obs_per_stock == expected_periods), 
    "out of", length(obs_per_stock), "\n")

# Check for any stocks with incomplete time series
if(any(obs_per_stock != expected_periods)) {
  cat("\nStocks with incomplete time series:\n")
  incomplete_stocks <- names(obs_per_stock)[obs_per_stock != expected_periods]
  print(data.frame(
    Stock = incomplete_stocks,
    Observations = as.numeric(obs_per_stock[incomplete_stocks]),
    Missing = expected_periods - as.numeric(obs_per_stock[incomplete_stocks])
  ))
}

# Display the first few rows of the melted data
head(volatility)

- Take logarithm of volatility

In [ ]:
# Create a copy of the volatility dataframe for log transformation
volatility_log <- volatility

# Check for non-positive values in the volatility column
non_positive_count <- sum(volatility_log$volatility <= 0, na.rm = TRUE)
if (non_positive_count > 0) {
    cat("Warning:", non_positive_count, "non-positive volatility values found.\n")
    cat("Adding a small constant before log transformation to handle zeros/negatives.\n")
    
    # Add a small constant to handle zeros and negative values
    min_positive <- min(volatility_log$volatility[volatility_log$volatility > 0], na.rm = TRUE)
    offset <- min_positive * 0.001  # Small fraction of the minimum positive value
    volatility_log$volatility <- volatility_log$volatility + offset
}

# Apply log transformation
volatility_log$volatility <- log(volatility_log$volatility)

# Verify the transformation
cat("Volatility summary before log transformation:\n")
print(summary(volatility$volatility))
cat("\nVolatility summary after log transformation:\n")
print(summary(volatility_log$volatility))

# Update the volatility dataframe with log-transformed values
volatility <- volatility_log

# Display the first few rows of the transformed dataframe
head(volatility)

- Remove stocks incurred extreme volatility within observation period

In [ ]:
# Load required libraries for normality tests and statistics
library(moments)  # For skewness and kurtosis functions

# Initialize an empty vector to store outlier stock IDs
outlier_stocks <- character(0)

# Function to measure distance from normal distribution
# Lower value = closer to normal distribution
normality_distance <- function(x) {
  # Calculate absolute skewness (0 for normal) and excess kurtosis (0 for normal)
  sk <- abs(skewness(x, na.rm = TRUE))
  kt <- abs(kurtosis(x, na.rm = TRUE) - 3)  # Normal kurtosis is 3
  
  # Combined distance metric
  return(sk + kt/2)  # Weighting kurtosis less as it's more sensitive
}

# Get unique year-month combinations
periods <- unique(volatility[, c("year", "month")])

# For each year-month period, transform distribution toward normal
for (i in 1:nrow(periods)) {
  curr_year <- periods$year[i]
  curr_month <- periods$month[i]
  
  # Get data for this period
  period_data <- volatility[volatility$year == curr_year & volatility$month == curr_month, ]
  
  # Skip if too few stocks for meaningful distribution shaping
  if (nrow(period_data) < 10) next
  
  # Calculate initial normality distance
  initial_distance <- normality_distance(period_data$volatility)
  
  # Prepare working copy of data
  working_data <- period_data
  period_stocks <- working_data$stkcd
  period_values <- working_data$volatility
  
  # Set maximum percentage of stocks to remove (to avoid over-pruning)
  max_remove_pct <- 0.1  # 10% maximum removal
  max_removals <- floor(length(period_stocks) * max_remove_pct)
  
  # Iteratively remove extreme values
  removals <- 0
  period_outliers <- character(0)
  
  while (removals < max_removals) {
    # Find current mean and standard deviation
    curr_mean <- mean(period_values, na.rm = TRUE)
    curr_sd <- sd(period_values, na.rm = TRUE)
    
    # Calculate z-scores to identify extremes
    z_scores <- abs((period_values - curr_mean) / curr_sd)
    
    # Find the most extreme value (highest z-score)
    max_idx <- which.max(z_scores)
    
    # Temporarily remove it
    test_values <- period_values[-max_idx]
    
    # Check if removal improves normality
    new_distance <- normality_distance(test_values)
    
    # Stop if no improvement or minimal improvement
    if (new_distance >= initial_distance || (initial_distance - new_distance) < 0.05) {
      break
    }
    
    # Otherwise, keep this removal
    period_outliers <- c(period_outliers, period_stocks[max_idx])
    period_stocks <- period_stocks[-max_idx]
    period_values <- test_values
    removals <- removals + 1
    initial_distance <- new_distance
  }
  
  # Add to our master list of outlier stocks
  outlier_stocks <- unique(c(outlier_stocks, period_outliers))
  
  # Report progress if significant removals made
  if (length(period_outliers) > 0) {
    cat(sprintf("Period %d-%02d: Removed %d stocks to improve normality\n", 
                curr_year, curr_month, length(period_outliers)))
  }
}

# Print the stocks to be removed
cat("Removing", length(outlier_stocks), "stocks to make distributions more normal\n")

# Create a filtered version of the volatility dataframe
volatility_filtered <- volatility[!volatility$stkcd %in% outlier_stocks, ]

# Update stock_id list to exclude outlier stocks
stock_id <- stock_id[!stock_id %in% outlier_stocks]

# Print summary statistics before and after filtering
cat("\nBefore filtering:", nrow(volatility), "observations,", length(unique(volatility$stkcd)), "stocks\n")
cat("After filtering:", nrow(volatility_filtered), "observations,", length(unique(volatility_filtered$stkcd)), "stocks\n")

# Update the main volatility dataframe
volatility <- volatility_filtered

# Display the first few rows of the filtered dataframe
head(volatility)

In [ ]:
# 1. Filter for quarter-end volatility (month is 3, 6, 9, or 12)
volatility_quarter_end <- volatility[volatility$month %in% c(3, 6, 9, 12), ]

# 2. Calculate quarterly average volatility for each stock
volatility_avg <- aggregate(
    volatility ~ stkcd + year + quarter,
    data = volatility,
    FUN = mean
)

# 3. Merge quarter-end data with quarterly average
volatility_quarterly <- merge(
    volatility_quarter_end,
    volatility_avg[, c("stkcd", "year", "quarter", "volatility")],
    by = c("stkcd", "year", "quarter"),
    all.x = TRUE,
    suffixes = c("_end", "_avg")
)

# 4. Create a clean dataframe with the required columns
volatility <- volatility_quarterly[, c("stkcd", "year", "quarter", "month", "volatility_end", "volatility_avg")]


# Report the dimensions of the new dataset
cat("New quarterly volatility dataset has", nrow(volatility), "observations for", 
        length(unique(volatility$stkcd)), "stocks\n")

# Display the first few rows of the updated volatility dataframe
head(volatility)

- Visualization

In [ ]:
# Load required libraries
library(ggplot2)
library(ggridges)
library(dplyr)

# Create a copy of the volatility dataframe for visualization
vol_viz <- volatility

# Create a proper year-quarter factor for chronological ordering
vol_viz$yearquarter <- paste0(vol_viz$year, "-Q", vol_viz$quarter)

# Create a chronologically ordered factor for the y-axis
unique_yearquarters <- unique(vol_viz$yearquarter[order(vol_viz$year, vol_viz$quarter)])
vol_viz$yearquarter <- factor(vol_viz$yearquarter, levels = unique_yearquarters)

# Ridge plot of volatility distributions over time
ggplot(vol_viz, aes(x = volatility_end, y = yearquarter, fill = ..x..)) +
  geom_density_ridges_gradient(scale = 3, rel_min_height = 0.01, alpha = 0.8) +
  scale_fill_viridis_c(name = "Volatility", option = "C") +
  labs(title = "Distribution of End-of-Quarter Stock Volatility Over Time",
       x = "Volatility (End of Quarter)",
       y = "Year-Quarter") +
  theme_ridges(font_size = 10, grid = TRUE) +
  theme(
    legend.position = "right",
    axis.text.y = element_text(size = 8, hjust = 1),
    plot.title = element_text(hjust = 0.5),
    panel.spacing.y = unit(0.5, "lines")
  )

# Create a summary of volatility measures for time series visualization
vol_summary <- vol_viz %>%
  group_by(year, quarter, yearquarter) %>%
  summarize(
    median_vol = median(volatility_end, na.rm = TRUE),
    q25 = quantile(volatility_end, 0.25, na.rm = TRUE),
    q75 = quantile(volatility_end, 0.75, na.rm = TRUE),
    .groups = "drop"
  )

# Convert yearquarter to a factor with same ordering
vol_summary$yearquarter <- factor(vol_summary$yearquarter, levels = unique_yearquarters)

# Time series plot showing median volatility and IQR
ggplot(vol_summary, aes(x = as.numeric(yearquarter), y = median_vol)) +
  geom_line(size = 1, color = "darkblue") +
  geom_ribbon(aes(ymin = q25, ymax = q75), alpha = 0.3, fill = "lightblue") +
  labs(title = "Median Stock Volatility Over Time (with IQR)",
       x = "Time", 
       y = "Volatility (End of Quarter)") +
  theme_minimal() +
  theme(
    plot.title = element_text(hjust = 0.5),
    axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)
  ) +
  # Add custom x-axis labels with appropriate gaps
  scale_x_continuous(
    breaks = seq(1, length(unique_yearquarters), by = 4),
    labels = unique_yearquarters[seq(1, length(unique_yearquarters), by = 4)]
  )

### Data Merging

- 1. Merge volatility with GFC indicators

In [ ]:
panel_data <- merge(
    volatility,
    gfc_indicators,
    by = c("year", "month", "quarter"),
    all.x = TRUE
)

# Show a sample of the final merged dataset
head(panel_data)

- 2. Merge with control variables

In [ ]:
panel_data <- merge(
    panel_data,
    control,
    by = c("stkcd", "year", "quarter", "month"),
    all.x = FALSE
)

# Display column names to verify what's available
cat("Columns in merged panel_data:\n")
print(names(panel_data))

# Create factor variables for fixed effects
panel_data$quarter <- factor(panel_data$quarter)  # Quarter fixed effects

# Check for missing data in key columns
missing_counts <- colSums(is.na(panel_data))
cat("Missing value counts in final panel dataset:\n")
print(missing_counts[missing_counts > 0])

# Show a sample of the final merged dataset
head(panel_data)

- 3. Merge with Capital Inflow

In [ ]:
# Merge panel_data with inflow data based on year and quarter
# Using inflow data read previously
panel_data <- merge(
    panel_data,
    inflow[, c("year", "quarter", "month", "inflow")],
    by = c("year", "quarter", "month"),
    all.x = TRUE
)

# Check for missing values in the inflow variable
cat("Missing values in inflow variable:", sum(is.na(panel_data$inflow)), "\n")

# Display dimensions of the enhanced panel dataset
cat("\nPanel dataset dimensions after adding inflow data: ", 
    nrow(panel_data), "observations,", ncol(panel_data), "variables\n")

# Display a sample of the merged data
head(panel_data)


- 4. Merge with EPU

In [ ]:
# Merge panel_data with EPU data based on year, quarter, and month
panel_data <- merge(
    panel_data,
    epu[, c("year", "quarter", "month", "log_epu")],
    by = c("year", "quarter", "month"),
    all.x = TRUE
)

# Check for missing values in the EPU variable
cat("Missing values in EPU variable:", sum(is.na(panel_data$log_epu)), "\n")

# Display dimensions of the enhanced panel dataset
cat("\nPanel dataset dimensions after adding EPU data: ", 
    nrow(panel_data), "observations,", ncol(panel_data), "variables\n")

# Display a sample of the merged data with EPU
head(panel_data)

- 5. Convert to panel

In [ ]:
library(plm)

# Create panel data structure
panel_data <- pdata.frame(panel_data, index = c("stkcd", "year", "quarter"))

# Basic panel information
cat("Panel Data Structure:\n")
cat("-------------------\n")
cat("Total observations:", nrow(panel_data), "\n")
cat("Number of stocks:", length(unique(panel_data$stkcd)), "\n")
cat("Number of time periods:", length(unique(paste(panel_data$year, panel_data$quarter))), "\n")

# Check if panel is balanced
pdim <- pdim(panel_data)
cat("\nPanel Dimensions:\n")
print(pdim)

if (pdim$balanced) {
    cat("\nThe panel is balanced.\n")
} else {
    cat("\nThe panel is unbalanced.\n")
    # Calculate distribution of observations per stock
    obs_per_stock <- table(index(panel_data, "id"))
    cat("Observation distribution per stock:\n")
    print(summary(as.numeric(obs_per_stock)))
}

# Display the first few rows
head(panel_data)
tail(panel_data)

# Regression

1. Main

- $\text{Volatility}_{i,t} = \alpha + \beta \text{GFC}_{t-1} + \sum \text{Control} + \sum \text{FE} + \epsilon_{i,t}$

|Model|Volatility|GFC (lagged 1 period)|Firm|Macro|FE(S)|FE(Q)|FE(P)|FE(I)| 
|:-|:-|:-|:-:|:-:|:-:|:-:|:-:|:-:|
|a|$\ln(\text{Volatility}_{\text{end of quarter }t})$|$\Delta\ln(\text{VIX}_{\text{end of quarter }t})$|||⚪|⚪|||
|a1|$\ln(\text{Volatility}_{\text{end of quarter }t})$|$\Delta\ln(\text{VIX}_{\text{end of quarter }t})$|⚪|⚪|⚪|⚪|||

2. Robustness Test

|Model|Volatility|GFC (lagged 1 period)|Firm|Macro|FE(S)|FE(Q)|FE(P)|FE(I)|
|:-|:-|:-|:-:|:-:|:-:|:-:|:-:|:-:|
|a2|$\ln(\overline{\text{Volatility}_{\text{quarter }t}})$|$\Delta\ln(\overline{\text{VIX}_{\text{quarter }t}})$|⚪|⚪|⚪|⚪|||
|a3|$\ln(\text{Volatility}_{\text{end of quarter }t})$|$\Delta\text{GFCF}_{\text{end of quarter }t}$|⚪|⚪|⚪|⚪|||
|a4|$\ln(\text{Volatility}_{\text{end of quarter }t})$|$\Delta\ln(\text{VIX}_{\text{end of quarter }t})$|⚪|⚪|⚪|⚪|⚪|⚪|

3. Channel Verification: `GFC` >> Channel >> `Volatility`

    1. $\text{Channel}_{i,t} = \alpha + \beta \text{GFC}_{t-1} + \sum \text{Control} + \sum \text{FE} + \epsilon_{i,t}$
    2. $\text{Volatility}_{i,t} = \alpha + \beta \text{Channel}_{t} + \sum \text{Control} + \sum \text{FE} + \epsilon_{i,t}$

|Model|Volatility|Channel|GFC (lagged 1 period)|Firm|Macro|FE(S)|FE(Q)|
|:-|:-|:-|:-|:-:|:-:|:-:|:-:|
|b1|$\ln(\text{Volatility}_{\text{end of quarter }t})$|`inflow`|$\Delta\ln(\text{VIX}_{\text{end of quarter }t})$|⚪|⚪|⚪|⚪|
|b2|$\ln(\text{Volatility}_{\text{end of quarter }t})$|`epu`|$\Delta\ln(\text{VIX}_{\text{end of quarter }t})$|⚪|⚪|⚪|⚪|

where:

||Channels|
|:-|:-:|
|`inflow`| |
|`epu`| |

and

||Control Variables|
|:-|:-:|
|Firm|`Size`, `Lev`, `ROA`, `BM`, `Board`, `Top1`|
|Macro|`GDP Growth Rate`, `Money Supply Growth Rate`|

also

||Fixed Effects|
|:-|:-:|
|FE(S)|`Individual (stock)`|
|FE(Q)|`Quarter`|
|FE(P)|`Province`|
|FE(I)|`Industry`|

## Main

### Model a

In [ ]:
library(plm)

# Run model with individual fixed effects and quarter dummies
model_a <- plm(volatility_end ~ log_diff_lag1 + I(log_diff_lag1^2) + I(log_diff_lag1^3) + quarter, 
             data = panel_data, 
             model = "within", 
             effect = "individual")

# Display regression results
summary(model_a)

# Get robust standard errors for both models
library(lmtest)
library(sandwich)
cat("\nRobust standard errors for two-way fixed effects model:\n")
coeftest(model_a, vcov = vcovHC(model_a, type = "HC1"))

### Model a1

#### Panel Regression

In [ ]:
library(plm)

# Run model with individual fixed effects and quarter dummies
model_a1 <- plm(volatility_end ~ log_diff_lag1 + I(log_diff_lag1^2) + I(log_diff_lag1^3) +  Size + Lev + ROA + BM + Board + Top1 + quarter + gdp_growth_rate + m2_growth_rate, 
             data = panel_data, 
             model = "within", 
             effect = "individual")

# Display regression results
summary(model_a1)

# Get robust standard errors for both models
library(lmtest)
library(sandwich)
cat("\nRobust standard errors for two-way fixed effects model:\n")
coeftest(model_a1, vcov = vcovHC(model_a1, type = "HC1"))

In [ ]:
par(bg="white", mar=c(5, 4, 4, 2) + 0.1)

# Extract coefficients for log_diff_lag1 and its higher order terms
coefficients <- coef(model_a1)
b1 <- coefficients["log_diff_lag1"]
b2 <- coefficients["I(log_diff_lag1^2)"]
b3 <- coefficients["I(log_diff_lag1^3)"]

# Use the range from the original gfc_indicators dataframe
min_value <- min(gfc_indicators$log_diff_lag1, na.rm = TRUE)
max_value <- max(gfc_indicators$log_diff_lag1, na.rm = TRUE)
log_diff_values <- seq(min_value, max_value, length.out = 100)

# Calculate predicted volatility for each value (partial effect, ignoring other variables)
predicted_volatility <- b1 * log_diff_values + 
                        b2 * log_diff_values^2 + 
                        b3 * log_diff_values^3

# Get the variance-covariance matrix for the coefficients
vcov_matrix <- vcovHC(model_a1, type = "HC1") # Using robust variance-covariance matrix

# Extract the rows/columns for our parameters of interest
param_indices <- which(names(coefficients) %in% c("log_diff_lag1", "I(log_diff_lag1^2)", "I(log_diff_lag1^3)"))
vcov_subset <- vcov_matrix[param_indices, param_indices]

# Calculate standard error for each prediction point
se_predictions <- numeric(length(log_diff_values))
for (i in 1:length(log_diff_values)) {
    # Create X matrix for this prediction point
    X_i <- c(log_diff_values[i], log_diff_values[i]^2, log_diff_values[i]^3)
    
    # Calculate variance of prediction
    se_predictions[i] <- sqrt(t(X_i) %*% vcov_subset %*% X_i)
}

# Calculate confidence intervals (95%)
confidence_level <- 0.95
t_value <- qt((1 + confidence_level) / 2, df = df.residual(model_a1))
upper_ci <- predicted_volatility + t_value * se_predictions
lower_ci <- predicted_volatility - t_value * se_predictions

# Create the plot
plot(log_diff_values, predicted_volatility, type = "l", lwd = 2, col = "blue",
     xlab = "Log Difference of VIX (lagged 1 period)",
     ylab = "Predicted Volatility",
     main = "Effect of VIX Changes on Stock Volatility",
     ylim = range(c(lower_ci, upper_ci)))

# Add confidence intervals
polygon(c(log_diff_values, rev(log_diff_values)), 
        c(lower_ci, rev(upper_ci)), 
        col = rgb(0, 0, 1, 0.2), border = NA)

# Add the CI lines
lines(log_diff_values, upper_ci, lty = 2, col = "blue")
lines(log_diff_values, lower_ci, lty = 2, col = "blue")

# Add a vertical line at x = 0 for reference
abline(v = 0, lty = 2, col = "darkgray")

# Add a grid for better readability
grid()

# Add a legend
legend("bottomleft", 
       legend = c("Predicted Effect", "95% Confidence Interval"),
       col = c("blue", rgb(0, 0, 1, 0.2)),
       lty = c(1, NA), 
       lwd = c(2, NA),
       fill = c(NA, rgb(0, 0, 1, 0.2)),
       border = c(NA, NA))

#### Testing

- Hausman

In [ ]:
# First estimate a random effects version of model_a1
model_a1_re <- plm(volatility_end ~ log_diff_lag1 + I(log_diff_lag1^2) + I(log_diff_lag1^3) +  
                   Size + Lev + ROA + BM + Board + Top1 + quarter + 
                   gdp_growth_rate + m2_growth_rate, 
                   data = panel_data, 
                   model = "random")

# Perform Hausman test
hausman_test <- phtest(model_a1, model_a1_re)

# Print the results
print(hausman_test)

- Residual Plots

In [ ]:
# Extract fitted values and residuals
fitted_values_model_a1 <- as.numeric(fitted(model_a1))
residuals_model_a1 <- as.numeric(residuals(model_a1))

- Fitted vs Residual Plot

In [ ]:
par(bg="white")

# Create a scatter plot of fitted values vs residuals
plot(fitted_values_model_a1, residuals_model_a1, 
    pch = 20, col = "darkblue",
    xlab = "Fitted Values", ylab = "Residuals",
    main = "Residual Diagnostic Plot for Model a1")

# Add a horizontal line at y = 0
abline(h = 0, col = "red", lwd = 2)

# Add a smoothed line to detect patterns
# Create a data frame for loess model
loess_data <- data.frame(x = fitted_values_model_a1, y = residuals_model_a1)
smooth_line <- loess(y ~ x, data = loess_data, span = 0.75)

# Sort the data for smooth line plotting
sorted_data <- loess_data[order(loess_data$x),]
# Predict using the sorted data
smooth_pred <- predict(smooth_line, newdata = sorted_data)

# Plot the smooth line
lines(sorted_data$x, smooth_pred, col = "cyan", lwd = 2)

# Add a legend
legend("topright", 
      legend = c("Residuals", "Reference Line", "Smooth Line"),
      col = c("darkblue", "red", "cyan"),
      pch = c(20, NA, NA), 
      lty = c(NA, 1, 1),
      lwd = c(NA, 2, 2))

# Add grid for better readability
grid()

- Q-Q Plot

In [ ]:
par(bg="white")
qqnorm(fitted_values_model_a1)
qqline(residuals_model_a1, col="red")

## Robustness Test

### Model a2

> Robustness verification of Model a1 by changing $\text{Volatility}$ and $\text{GFC}$ from quarter-end data to quarterly-averaged data.

#### Panel Regression

In [ ]:
library(plm)

# Run model with individual fixed effects and quarter dummies
model_a2 <- plm(volatility_avg ~ log_diff_avg_lag1 + I(log_diff_avg_lag1^2) + I(log_diff_avg_lag1^3) + Size + Lev + ROA + BM + Board + Top1 + quarter + gdp_growth_rate + m2_growth_rate, 
             data = panel_data, 
             model = "within", 
             effect = "individual")

# Display regression results
summary(model_a2)

# Get robust standard errors for both models
library(lmtest)
library(sandwich)
cat("\nRobust standard errors for two-way fixed effects model:\n")
coeftest(model_a2, vcov = vcovHC(model_a2, type = "HC1"))

#### Testing

- Residual Plots

In [ ]:
# Extract fitted values and residuals
fitted_values_model_a2 <- as.numeric(fitted(model_a2))
residuals_model_a2 <- as.numeric(residuals(model_a2))

- Fitted vs Residual Plot

In [ ]:
par(bg="white")

# Create a scatter plot of fitted values vs residuals
plot(fitted_values_model_a2, residuals_model_a2, 
    pch = 20, col = "darkblue",
    xlab = "Fitted Values", ylab = "Residuals",
    main = "Residual Diagnostic Plot for Model a1")

# Add a horizontal line at y = 0
abline(h = 0, col = "red", lwd = 2)

# Add a smoothed line to detect patterns
# Create a data frame for loess model
loess_data <- data.frame(x = fitted_values_model_a2, y = residuals_model_a2)
smooth_line <- loess(y ~ x, data = loess_data, span = 0.75)

# Sort the data for smooth line plotting
sorted_data <- loess_data[order(loess_data$x),]
# Predict using the sorted data
smooth_pred <- predict(smooth_line, newdata = sorted_data)

# Plot the smooth line
lines(sorted_data$x, smooth_pred, col = "cyan", lwd = 2)

# Add a legend
legend("topright", 
      legend = c("Residuals", "Reference Line", "Smooth Line"),
      col = c("darkblue", "red", "cyan"),
      pch = c(20, NA, NA), 
      lty = c(NA, 1, 1),
      lwd = c(NA, 2, 2))

# Add grid for better readability
grid()

- Q-Q Plot

In [ ]:
par(bg="white")
qqnorm(fitted_values_model_a2)
qqline(residuals_model_a2, col="red")

### Model a3

> Robustness verification of Model a1 by changing the proxy for $\text{GFC}$ from VIX-based data to GFCF.

#### Panel Regression

In [ ]:
library(plm)

# Run model with individual fixed effects and quarter dummies
model_a3 <- plm(volatility_end ~ diff_gfcf_lag1 + I(diff_gfcf_lag1^2) + I(diff_gfcf_lag1^3) + Size + Lev + ROA + BM + Board + Top1 + quarter + gdp_growth_rate + m2_growth_rate, 
             data = panel_data, 
             model = "within", 
             effect = "individual")

# Display regression results
summary(model_a3)

# Get robust standard errors for both models
library(lmtest)
library(sandwich)
cat("\nRobust standard errors for two-way fixed effects model:\n")
coeftest(model_a3, vcov = vcovHC(model_a3, type = "HC1"))

#### Testing

- Residual Plots

In [ ]:
# Extract fitted values and residuals
fitted_values_model_a3 <- as.numeric(fitted(model_a3))
residuals_model_a3 <- as.numeric(residuals(model_a3))

- Fitted vs Residual Plot

In [ ]:
par(bg="white")

# Create a scatter plot of fitted values vs residuals
plot(fitted_values_model_a3, residuals_model_a3, 
    pch = 20, col = "darkblue",
    xlab = "Fitted Values", ylab = "Residuals",
    main = "Residual Diagnostic Plot for Model a1")

# Add a horizontal line at y = 0
abline(h = 0, col = "red", lwd = 2)

# Add a smoothed line to detect patterns
# Create a data frame for loess model
loess_data <- data.frame(x = fitted_values_model_a2, y = residuals_model_a3)
smooth_line <- loess(y ~ x, data = loess_data, span = 0.75)

# Sort the data for smooth line plotting
sorted_data <- loess_data[order(loess_data$x),]
# Predict using the sorted data
smooth_pred <- predict(smooth_line, newdata = sorted_data)

# Plot the smooth line
lines(sorted_data$x, smooth_pred, col = "cyan", lwd = 2)

# Add a legend
legend("topright", 
      legend = c("Residuals", "Reference Line", "Smooth Line"),
      col = c("darkblue", "red", "cyan"),
      pch = c(20, NA, NA), 
      lty = c(NA, 1, 1),
      lwd = c(NA, 2, 2))

# Add grid for better readability
grid()

- Q-Q Plot

In [ ]:
par(bg="white")
qqnorm(fitted_values_model_a3)
qqline(residuals_model_a3, col="red")

### Model a4

> Robustness verification of Model a1 by controlling industrial and provincial fixed effects.

#### Panel Regression

In [ ]:
library(plm)

# Run model with individual fixed effects and quarter dummies
model_a4 <- plm(volatility_end ~ log_diff_avg_lag1 + I(log_diff_avg_lag1^2) + I(log_diff_avg_lag1^3) + Size + Lev + ROA + BM + Board + Top1 + quarter + ind + gdp_growth_rate + m2_growth_rate, 
             data = panel_data, 
             model = "within", 
             effect = "individual")

# Display regression results
summary(model_a4)

# Get robust standard errors for both models
library(lmtest)
library(sandwich)
cat("\nRobust standard errors for two-way fixed effects model:\n")
coeftest(model_a4, vcov = vcovHC(model_a4, type = "HC1"))

## Channel Verification

### Model b1

> `GFC` >> `International Capital Inflow` >> `Volatility`

- `GFC` >> `International Capital Inflow`

In [ ]:
library(plm)

# Run model with individual fixed effects and quarter dummies
model_b1_1 <- plm(inflow ~ log_diff_lag1 + I(log_diff_lag1^2) + I(log_diff_lag1^3) + Size + Lev + ROA + BM + Board + Top1 + quarter + gdp_growth_rate + m2_growth_rate, 
             data = panel_data, 
             model = "within", 
             effect = "individual")

# Display regression results
summary(model_b1_1)

# Get robust standard errors for both models
library(lmtest)
library(sandwich)
cat("\nRobust standard errors for two-way fixed effects model:\n")
coeftest(model_b1_1, vcov = vcovHC(model_b1_1, type = "HC1"))

In [ ]:
par(bg="white", mar=c(5, 4, 4, 2) + 0.1)

# Extract coefficients for log_diff_lag1 and its higher order terms
coefficients <- coef(model_b1_1)
b1 <- coefficients["log_diff_lag1"]
b2 <- coefficients["I(log_diff_lag1^2)"]
b3 <- coefficients["I(log_diff_lag1^3)"]

# Use the range from the original gfc_indicators dataframe
min_value <- min(gfc_indicators$log_diff_lag1, na.rm = TRUE)
max_value <- max(gfc_indicators$log_diff_lag1, na.rm = TRUE)
log_diff_values <- seq(min_value, max_value, length.out = 100)

# Calculate predicted inflow for each value (partial effect, ignoring other variables)
predicted_inflow <- b1 * log_diff_values + 
                    b2 * log_diff_values^2 + 
                    b3 * log_diff_values^3

# Get the variance-covariance matrix for the coefficients
vcov_matrix <- vcovHC(model_b1_1, type = "HC1") # Using robust variance-covariance matrix

# Extract the rows/columns for our parameters of interest
param_indices <- which(names(coefficients) %in% c("log_diff_lag1", "I(log_diff_lag1^2)", "I(log_diff_lag1^3)"))
vcov_subset <- vcov_matrix[param_indices, param_indices]

# Calculate standard error for each prediction point
se_predictions <- numeric(length(log_diff_values))
for (i in 1:length(log_diff_values)) {
    # Create X matrix for this prediction point
    X_i <- c(log_diff_values[i], log_diff_values[i]^2, log_diff_values[i]^3)
    
    # Calculate variance of prediction
    se_predictions[i] <- sqrt(t(X_i) %*% vcov_subset %*% X_i)
}

# Calculate confidence intervals (95%)
confidence_level <- 0.95
t_value <- qt((1 + confidence_level) / 2, df = df.residual(model_b1_1))
upper_ci <- predicted_inflow + t_value * se_predictions
lower_ci <- predicted_inflow - t_value * se_predictions

# Create the plot
plot(log_diff_values, predicted_inflow, type = "l", lwd = 2, col = "blue",
     xlab = "Log Difference of VIX (lagged 1 period)",
     ylab = "Predicted Inflow",
     main = "Effect of VIX Changes on Fund Inflow",
     ylim = range(c(lower_ci, upper_ci)))

# Add confidence intervals
polygon(c(log_diff_values, rev(log_diff_values)), 
        c(lower_ci, rev(upper_ci)), 
        col = rgb(0, 0, 1, 0.2), border = NA)

# Add the CI lines
lines(log_diff_values, upper_ci, lty = 2, col = "blue")
lines(log_diff_values, lower_ci, lty = 2, col = "blue")

# Add a vertical line at x = 0 for reference
abline(v = 0, lty = 2, col = "darkgray")

# Add a grid for better readability
grid()

# Add a legend
legend("bottomleft", 
       legend = c("Predicted Effect", "95% Confidence Interval"),
       col = c("blue", rgb(0, 0, 1, 0.2)),
       lty = c(1, NA), 
       lwd = c(2, NA),
       fill = c(NA, rgb(0, 0, 1, 0.2)),
       border = c(NA, NA))

- `International Capital Inflow` >> `Volatility`

In [ ]:
library(plm)

# Run model with individual fixed effects and quarter dummies
model_b1_2 <- plm(volatility_end ~ inflow +  Size + Lev + ROA + BM + Board + Top1 + quarter + gdp_growth_rate + m2_growth_rate, 
             data = panel_data, 
             model = "within", 
             effect = "individual")

# Display regression results
summary(model_b1_2)

# Get robust standard errors for both models
library(lmtest)
library(sandwich)
cat("\nRobust standard errors for two-way fixed effects model:\n")
coeftest(model_b1_2, vcov = vcovHC(model_b1_2, type = "HC1"))

In [ ]:
par(bg="white", mar=c(5, 4, 4, 2) + 0.1)

# Extract coefficient for inflow
coefficient <- coef(model_b1_2)["inflow"]
vcov_matrix <- vcovHC(model_b1_2, type = "HC1") 
se <- sqrt(vcov_matrix["inflow", "inflow"])

# Use the range from panel_data
min_value <- quantile(panel_data$inflow, 0.05, na.rm = TRUE)
max_value <- quantile(panel_data$inflow, 0.95, na.rm = TRUE)
inflow_values <- seq(min_value, max_value, length.out = 100)

# Calculate predicted change in volatility across the range
effect_values <- coefficient * inflow_values

# Calculate confidence intervals
t_value <- qt(0.975, df = df.residual(model_b1_2))
ci_width <- t_value * se * abs(inflow_values)
upper_ci <- effect_values + ci_width
lower_ci <- effect_values - ci_width

# Create the plot
plot(inflow_values, effect_values, type = "l", lwd = 2, col = "blue",
    xlab = "Fund Inflow",
    ylab = "Effect on Log Volatility",
    main = "Effect of Fund Inflow on Stock Volatility",
    ylim = range(c(lower_ci, upper_ci, 0)))  # Include 0 in range

# Add confidence intervals
polygon(c(inflow_values, rev(inflow_values)), 
       c(lower_ci, rev(upper_ci)), 
       col = rgb(0, 0, 1, 0.2), border = NA)

# Add the CI lines
lines(inflow_values, upper_ci, lty = 2, col = "blue")
lines(inflow_values, lower_ci, lty = 2, col = "blue")

# Add a horizontal line at y = 0 for reference
abline(h = 0, lty = 2, col = "red")

# Add a grid for better readability
grid()

# Add a legend
legend("bottomleft", 
      legend = c("Marginal Effect", "95% Confidence Interval", "Zero Reference"),
      col = c("blue", rgb(0, 0, 1, 0.2), "red"),
      lty = c(1, NA, 2), 
      lwd = c(2, NA, 1),
      fill = c(NA, rgb(0, 0, 1, 0.2), NA),
      border = c(NA, NA, NA))

# Add p-value information
p_value <- 2 * pt(-abs(coefficient/se), df = df.residual(model_b1_2))
p_text <- if (p_value < 0.001) "p < 0.001" else paste("p =", round(p_value, 3))
mtext(paste("Coefficient =", round(coefficient, 4), ",", p_text), 
     side = 3, line = 0.5, col = "darkblue", cex = 0.8)

### Model b2

> `GFC` >> `Policy Uncertainty` >> `Volatility`

- `GFC` >> `Policy Uncertainty`

In [ ]:
library(plm)

# Run model with individual fixed effects and quarter dummies
model_b2_1 <- plm(log_epu ~ log_diff_lag1 + I(log_diff_lag1^2) + I(log_diff_lag1^3) + Size + Lev + ROA + BM + Board + Top1 + quarter + gdp_growth_rate + m2_growth_rate, 
             data = panel_data, 
             model = "within", 
             effect = "individual")

# Display regression results
summary(model_b2_1)

# Get robust standard errors for both models
library(lmtest)
library(sandwich)
cat("\nRobust standard errors for two-way fixed effects model:\n")
coeftest(model_b2_1, vcov = vcovHC(model_b2_1, type = "HC1"))

- `Policy Uncertainty` >> `Volatility`

In [ ]:
library(plm)

# Run model with individual fixed effects and quarter dummies
model_b2_2 <- plm(volatility_end ~ log_epu +  Size + Lev + ROA + BM + Board + Top1 + quarter + gdp_growth_rate + m2_growth_rate, 
             data = panel_data, 
             model = "within", 
             effect = "individual")

# Display regression results
summary(model_b2_2)

# Get robust standard errors for both models
library(lmtest)
library(sandwich)
cat("\nRobust standard errors for two-way fixed effects model:\n")
coeftest(model_b2_2, vcov = vcovHC(model_b2_2, type = "HC1"))